<a href="https://colab.research.google.com/github/sujeet311/Quantuma_Boosting/blob/main/quantum_boosting(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
import time
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, roc_curve,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install dwave.system

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 2.8 MB/s 
     |████████████████████████████████| 11.8 MB 20.9 MB/s 
     |████████████████████████████████| 5.3 MB 28.2 MB/s 
     |████████████████████████████████| 700 kB 59.1 MB/s 
     |████████████████████████████████| 110 kB 63.7 MB/s 
     |████████████████████████████████| 966 kB 31.6 MB/s 
     |████████████████████████████████| 97 kB 5.7 MB/s 
     |████████████████████████████████| 44 kB 1.8 MB/s 
     |████████████████████████████████| 246 kB 59.9 MB/s 


In [ ]:
# import necessary packages

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from sklearn.impute import SimpleImputer
from qboost import WeakClassifiers, QBoostClassifier


In [ ]:
from numpy import loadtxt
X_train = loadtxt('/content/drive/MyDrive/code thesis/train_test/X_train.csv',delimiter=',')
y_train = loadtxt('/content/drive/MyDrive/code thesis/train_test/y_train.csv',delimiter=',')
X_test = loadtxt('/content/drive/MyDrive/code thesis/train_test/X_test.csv',delimiter=',')
y_test = loadtxt('/content/drive/MyDrive/code thesis/train_test/y_test.csv',delimiter=',')

In [ ]:
y_train = (2* y_train )-1
y_test = (2* y_test) -1

In [ ]:
# scaling data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
    # define sampler
endpoint = 'https://cloud.dwavesys.com/sapi'
token = 'DEV-a71a4e9dc32bb8967a50c5470614d84b8ca0e842'
solver = 'DW_2000Q_6'
dwave_sampler = DWaveSampler(endpoint=endpoint, token=token, solver=solver)
emb_sampler = EmbeddingComposite(dwave_sampler)
lmd = 1.0

In [ ]:
# define parameters used in this function
NUM_READS = 1000
NUM_WEAK_CLASSIFIERS = 50
TREE_DEPTH = 2
DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 'num_spin_reversal_transforms': 10,
                 'postprocess': 'optimization',
                 }
emb_sampler = EmbeddingComposite(dwave_sampler)


In [ ]:
QBC = QBoostClassifier(n_estimators=20, max_depth=2)
t=time.time()
QBC.fit(X_train, y_train,emb_sampler, lmd=lmd, **DW_PARAMS)
y_pred = QBC.predict(X_test)
print(accuracy_score(y_train, QBC.predict(X_train)))

QBC_acc = accuracy_score(y_test, y_pred)
print(QBC_acc)
# confusion matrix
target_names = ['0','1']
cm=confusion_matrix(y_test, y_pred)
# classification report

print(classification_report(y_test, y_pred,target_names=target_names))
elapsed_ab = time.time()-t
elapsed_ab



0.7259441363775002
0.725346617850069
              precision    recall  f1-score   support

           0       0.73      0.71      0.72     58030
           1       0.72      0.74      0.73     58597

    accuracy                           0.73    116627
   macro avg       0.73      0.73      0.73    116627
weighted avg       0.73      0.73      0.73    116627



17.043071746826172

In [ ]:
from joblib import Parallel, delayed
import joblib
# Save the model as a pickle in a file
joblib.dump(QBC, '/content/drive/MyDrive/QC Code/model/QBC.pkl')

['/content/drive/MyDrive/QC Code/model/QBC.pkl']